### Contributor: Sudesh Kumar Santhosh Kumar
- Email: sudeshkumar.santhoshkumar@lexisnexis.com
- Date: 21st June, 2023

Description:  This Notebook will show you how to fine-tune BERT for multi-class text classification tasks, meaning that there are more than just two classes in the dataset. We'll be using the 20 Newsgroups dataset, which is a test classification dataset with 20 different classes. 

**Multi-Label vs. Multi-Class**

It is important to distinguish between "Multi-Class" and "Multi-Label" as they pertain to different types of tasks and are handled differently.

In "Multi-Label" classification, documents can be assigned multiple labels. This means that a document may have several appropriate tags or categories associated with it. For instance, consider a system that suggests tags for documents where multiple tags may be relevant.

On the other hand, "Multi-Class" classification applies when each document is assigned only one category. If there are only two possible categories, it is known as "Binary" classification. If there are more than two categories, it is referred to as "Multi-Class" classification. In this scenario, each document is assigned to a single category, without the possibility of multiple labels.

By understanding these distinctions, we can appropriately handle and address the requirements of each type of classification task.

# Part I - Installation, Loading Dataset and Tokenization
---------------------------------

## S1. Setting up the Notebook


### 1.1. Basic Pre-requisites and Installations

If you're using this notebook on Google Colab, Don't forget to uncomment the following cell to install transformers library into the notebook. If you're using this notebook on your local machine through VS Code or Jupyter notebook, you need to do a "pip install -r requirements.txt" before running the following set of code blocks.

In [ ]:
# !pip install transformers

### 1.2. Importing all necessary libraries

In [ ]:
import textwrap
import random
import csv
import os
import time
import datetime

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

import torch
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import get_linear_schedule_with_warmup
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig

from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics




### 1.3. Connecting to GPU 

In [ ]:
# Checking if there is a GPU Available.
if torch.cuda.is_available():

    device = torch.device(("cuda"))
    print(f"There are {torch.cuda.device_count()} available.")
    print(f"We will use the GPU: {torch.cuda.get_device_name(0)}")

else:
    print("No GPU available, using the CPU!")
    device = torch.device("cpu")

## S2. Load Dataset from scikit-learn and Inspect it

### 2.1. import dataset from scikit-learn

The 20 Newsgroups dataset is old (from the 1990s?), and contains what today we would probably call "forum posts", coming from 20 different subtopics. 

The 20 categories (or) classes are as follows:

* comp.graphics
* comp.os.ms-windows.misc
* comp.sys.ibm.pc.hardware
* comp.sys.mac.hardware
* comp.windows.x	
* rec.autos
* rec.motorcycles
* rec.sport.baseball
* rec.sport.hockey	
* sci.crypt
* sci.electronics
* sci.med
* sci.space
* misc.forsale	
* talk.politics.misc
* talk.politics.guns
* talk.politics.mideast	
* talk.religion.misc
* alt.atheism
* soc.religion.christian

Scikit-learn includes some nice helper functions for retrieving the 20 Newsgroups dataset (from their example [here](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html)). We'll use them below to retrieve the dataset.


In [ ]:
# Removing the meta-data from the train and test subsets.
train = fetch_20newsgroups(subset='train',
                           remove=('headers', 'footers', 'quotes'))

test = fetch_20newsgroups(subset='test',
                           remove=('headers', 'footers', 'quotes'))

### 2.2. Inspecting the data

In [ ]:
# Wrap text to 80 characters.
wrapper = textwrap.TextWrapper(width=80) 

# Randomly choose some examples.
for i in range(10):
    
    # Choose a random sample by index.
    j = random.choice(range(len(train.data)))
    
    # Get the text as 'x' and the label integer as 'y'.
    x = train.data[j]
    y = train.target[j]

    # Print out the name of the category and the text.
    print('')
    print('========', train.target_names[y], '========')
    print(wrapper.fill(x))
    print('')


### 2.3. Exploratory Data Analysis and Checking for imbalances in the dataset.

In [ ]:
print(f"Total number of Data-points in the Train Data: {len(train.data)}")
print(f"Total number of data-points in the Test Data: {len(test.data)}")
print(f"Total number of classes in the Data-set, C = {len(np.unique(np.array(train.target)))}")

In [ ]:

sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (10,5)


# Plot the number of tokens of each length.
sns.countplot(x=train.target)
plt.title('Class Distribution')
plt.xlabel('Category')
plt.ylabel('# of Training Samples')


Except class 19, which corresponds to 'talk.religion.misc', every other class is well balanced.

In [ ]:
train.target_names[19]

## S3. Tokenization using BertTokenizer.

### 3.1 Loading the BertTokenizer.

To feed our text to BERT, it must be split into tokens, and then these tokens must be mapped to their index in the tokenizer vocabulary. The tokenization must be performed by the tokenizer included with BERT--the below cell will download this for us. We'll be using the "uncased" version here.

In [ ]:
# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
# Sample Toeknization Example.
# Print the original sentence.
print(' Original: ', train.data[0])
print()


# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(train.data[0]))
print()

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(train.data[0])))

In [ ]:
print(' Original: ', len(list(train.data[0])))

# Print the sentence split into tokens.
print('Tokenized: ', len(tokenizer.tokenize(train.data[0])))

# Print the sentence mapped to token ids.
print('Token IDs: ', len(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(train.data[0]))))

### 3.2. Tokenize All Samples

The tokenizer.encode function combines multiple steps for us:

* Split the sentence into tokens.
* Add the special [CLS] and [SEP] tokens.
* Map the tokens to their IDs.

In order to explore the distribution of sequence lengths, we will not perform any truncation here. Unfortunately, this results in the tokenizer spitting out a warning for every sample that's longer than 512 tokens. We'll just have to ignore those for now!

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

# Record the length of each sequence (in terms of BERT tokens).
lengths = []

print('Tokenizing the input sentences in train data (11314 examples)')

# For every sentence...
for sentence in train.data:

    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sentence,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        #max_length = 512,          # Truncate all sentences.                        
                        #return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.
    input_ids.append(encoded_sent)

    # Record the non-truncated length.
    lengths.append(len(encoded_sent))

print('Tokenization Process Completed.')
print(f"Tokenized {len(train.data)} sentences from the Train Data!")

### 3.3. Understanding the distribution of sentence lengths in the Train Data.

In [ ]:
print(f"Min length: {min(lengths)} tokens")
print(f'Max length: {max(lengths)} tokens')
print(f'Median length: {np.median(lengths)} tokens')

### 3.4. Analysing the percentage of data over the 512 limit

In [ ]:
num_over = 0

# For all of the length values...
for length in lengths:
    # Tally if it's over 512.
    if length > 512:
        num_over += 1

print('{:,} of {:,} comments will be truncated ({:.2%})'.format(num_over, len(lengths), float(num_over) / float(len(lengths))))

The BERT-base-uncased model indeed uses an embedding layer with weights of dimension 30522 x 768 for the wordpiece vocabulary.

In BERT, the first embedding layer is known as the token embedding layer, and it maps the input wordpiece IDs to their corresponding embedding vectors. The vocabulary size in BERT-base-uncased is 30522, meaning it can handle up to 30522 different wordpiece tokens.

Each wordpiece token in the vocabulary is associated with a 768-dimensional embedding vector. Therefore, the embedding layer in BERT-base-uncased has weights with dimensions of 30522 x 768, where each row corresponds to the embedding vector of a specific wordpiece token in the vocabulary.

Before converting text to wordpiece tokens in BERT, there is typically a limit on the maximum number of tokens in a sentence. This limit is often referred to as the "maximum sequence length" or MAX_LEN.

In BERT, the maximum sequence length is determined based on the architecture and configuration of the specific model being used. The most common maximum sequence length used in BERT models is 512 tokens.

If a sentence has more than the maximum sequence length, it needs to be truncated or split into smaller parts to fit within the limit. Truncation involves discarding tokens from the end of the sentence, while splitting involves dividing the sentence into multiple segments or chunks.

### 3.5. Truncating the longer sentences and Perform Tokenization

Given the higher percentage of longer-than-512-tokens samples, I thought it might make sense to try a maximum length of 512 for this example. However, with batch_size = 32 and max_len = 512, I ran into a GPU out-of-memory error on the first iteration of the training loop!

After performing some analysis and a number of experiment,

* If the combination of your batch_size and max_len are too high, then you can easily run out of memory (even on a Tesla K80 with 12GB!) when fine-tuning BERT-base. For example, with batch_size = 16, a max_len = ~400 is about the limit on a Tesla K80.
* Reducing batch_size reduces memory consumption, but at the cost of slower training, and finding the best choice of batch_size appears to be important for accuracy.
* Reducing max_len reduces memory consumption and speeds up training, but has a substantial impact on BERT's accuracy on this benchmark.

In [ ]:
max_len = 300

In [ ]:
def my_tokenize(comments, max_len):
    '''
    Tokenize a dataset of comments.

    Parameters:
      `comments` - List of comments, represented as strings.
        `labels` - List of integer labels for the corresponding comments.
       `max_len` - Truncate all of the comments down to this length.
    
    Returns:
      `input_ids` - All of the comments represented as lists of token IDs,
                    padded out to `max_len`, and cast as a PyTorch tensor.
         `labels` - The labels for the corresponding comments, formatted as 
                    a PyTorch tensor.
      `attention_masks` - PyTorch tensor with the same dimensions as
                          `input_ids`. For each token, simply indicates whether
                           it is padding or not.
    '''
    # Tokenize all of the sentences and map the tokens to thier word IDs.
    input_ids_truncated = []
    attention_masks = []

    print('Tokenizing {:,} comments...'.format(len(comments)))

    # For every comment ("sentence")...
    for sent in comments:

        # Report progress.
        if ((len(input_ids_truncated) % 500) == 0):
            print('  Tokenized {:,} comments.'.format(len(input_ids_truncated)))

        # `encode_plus` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to `max_length`
        #   (6) Create attention masks for [PAD] tokens.
        encoded_dict = tokenizer.encode_plus(
                            sent,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = max_len,      # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                    )
        
        # Add the encoded sentence to the list.    
        input_ids_truncated.append(encoded_dict['input_ids'])
        
        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(encoded_dict['attention_mask'])

    # Convert the lists into tensors.
    input_ids_truncated = torch.cat(input_ids_truncated, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    print("Completed the tokenization process for all the comments passed!")
    return (input_ids_truncated, labels, attention_masks)


In [ ]:
# Tokenize our entire training set.
(train_input_ids,  
 train_attention_masks) = my_tokenize(train.data, max_len = max_len)

Let's test a sample sentence

In [ ]:
# Print sentence 0, now as a list of IDs.
print('Original: ', train.data[0])
print()
print('Token IDs:', train_input_ids[0])

Comparing the lengths of the input tokens before Padding and Truncation and the input tokens after padding and truncation.

In [ ]:
for i in range(len(train.data)):
    if len(input_ids[i]) < 10:
        print(f"Length of original sentence: {len(train.data[i])}")
        print(f"Length of input tokens before truncation / Padding: {len(input_ids[i])}")
        print(f"Length of train input ids after truncation: {len(train_input_ids[i])}")

# Part II - Fine Tuning of BERT
---------------------------

## S4. Training the Classifier

### 4.1. Loading the pre-trained BERT Model

Since it is a multi-class classification task, the output layer will be a Softmax Layer.

In [ ]:
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 20, # For our 20 newsgroups!
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

desc = model.to(torch.device("cpu"))

# Tell pytorch to run this model on the GPU.
# desc = model.cuda()

## In the below cell, I've printed out the names and dimensions of the weights for:

* The embedding layer.
* The first of the twelve transformers.
* The output layer.

In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

We can see that for the previous example, the output layer had 2 x 768 weights and 1 bias term. 
Whereas for the Multi-class example, we can see the O/P layer has 20 x 768 weights and 20 bias terms.

### 4.2. Training and validation Split

In [ ]:
train_labels = torch.tensor(train.target)
train_labels = train_labels.to(torch.long)


# Use 90% for training and 10% for validation.
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(train_input_ids, train_labels,
                                                            random_state=2018, test_size=0.1)
# Do the same for the masks.
train_masks, validation_masks, _, _ = train_test_split(train_attention_masks, torch.tensor(train.target),
                                             random_state=2018, test_size=0.1)

print(f"Total number of Training Samples: {len(train_inputs)}")
print(f"Total number of Validation Samples: {len(validation_inputs)}")

### 4.3. Batch Size and Data-loaders

### We'll also create an iterator for our dataset using the torch DataLoader class. This helps save on memory during training because, unlike a for loop, with an iterator the entire dataset does not need to be loaded into memory.

In [ ]:
# The DataLoader needs to know our batch size for training, so we specify it
# here.
# For fine-tuning BERT on a specific task, the authors recommend a batch size of
# 16 or 32.

batch_size = 16

# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


### 4.4. Optimizer, Epochs and Learning Rate Scheduler

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)


### 4.5 Helper Functions

In [ ]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

### 4.6. Training Loop!

In [ ]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 50 batches.
        if step % 50 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        result = model(b_input_ids, 
                       token_type_ids = None, 
                       attention_mask = b_input_mask, 
                       labels = b_labels,
                       return_dict = True)
        
        loss = result.loss

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            result = model(b_input_ids, 
                           token_type_ids = None, 
                           attention_mask = b_input_mask,
                           labels = b_labels,
                           return_dict = True)
            
            loss = result.loss
            logits = result.logits
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    validation_time = format_time(time.time() - t0)
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

### 4.6. Visualization of Train and Validation Loss Vs. Epochs

In [ ]:
import matplotlib.pyplot as plt
% matplotlib inline

import seaborn as sns

# Use plot styling from seaborn.
sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")

# Label the plot.
plt.title("Training & Validation loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks([1, 2, 3, 4])

plt.show()